In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import re

# Get all jsons
all_pages = []
all_ids = []

url = "https://www.mara.gov.au/api/agentsearch?DelimitedStartWithLetterFilter%5BFieldName%5D=Name.FamilyName&DelimitedStartWithLetterFilter%5BLetterString%5D=&DelimitedStartWithLetterFilter%5BLabel%5D=Show+All&DelimitedStartWithLetterFilter%5BIsSelected%5D=false&DelimitedStartWithLetterFilter%5BContainsData%5D=true&Keyword=&Location=Australia&BusinessName=&IsNoFee=&IsPractitioner=&AgentFamilyName=&AgentGivenName=&AgentName=&AgentMARN=&SortInfo%5BSortField%5D=&SortInfo%5BIsAscending%5D=false&PagingInfo%5BPageIndex%5D=0&PagingInfo%5BPageSize%5D=8915"

pages = requests.get(url)
response = pages.json()['Result']

In [ ]:
# Distract all unique ids from json files

contact_ids = []
for i in range(len(response)):
    contact_ids.append(response[i]['ContactId'])
contact_ids = list(set(contact_ids))

In [ ]:
# Find all agent names
# 如果要加入公司别的信息在这里加即可

name_ids = []
full_names = []
for i in range(len(response)):
    full_name = response[i]['Name']['FullName']
    company_id = response[i]['ContactId']
    full_names.append(full_name)
    name_ids.append(company_id)

In [ ]:
# Find all agent emails

agent_ids = []
agent_emails = []
for index in range(len(contact_ids)):
    url_1 = 'https://www.mara.gov.au/search-the-register-of-migration-agents/registered-migration-agent-details/?id='+contact_ids[index]
    result =  requests.get(url_1)
    soup = BeautifulSoup(result.content, 'lxml')
    agent_email = soup.find_all('a',href = re.compile('@'))
    if len(agent_email) == 0:
        continue
        
    email = str(agent_email[0]).split('"')[1].split('mailto:')[1]
    agent_ids.append(contact_ids[index])
    agent_emails.append(email)

In [ ]:
# Generate email form and name form

data_1 = {
    'id': agent_ids,
    'email': agent_emails
}

data_2 = {
    'id': name_ids,
    'name': full_names
}

email_form = pd.DataFrame(data_1)
name_form = pd.DataFrame(data_2)
name_form.drop_duplicates(inplace = True)

In [ ]:
# inner merge forms and output csv

final_form = pd.merge(email_form, name_form, how='inner', on=['id'])
final_form[['name','email']].to_csv('Emails of agents.csv', index=False)